In [7]:
from flask import Flask,request
from flask_restful import Resource, Api
from flask_jwt import JWT,jwt_required
from security import authenticate,identity

app = Flask(__name__)
api = Api(app)
app.secret_key = 'jose'
jwt = JWT(app,authenticate,identity) 

items = [
    {
        "name":"bag",
        "price":24
    }
]

class Item(Resource):
    @jwt_required()
    def get(self,name):
        data=next(filter(lambda x: x["name"]==name,items),None)
        
        return data,200 if data is not None else 404
    def post(self,name):
        if next(filter(lambda x: x["name"]==name,items),None):
            return {"message":f"The item with name {name} alredy exists"},400
        item = {"name":name,"price":54}
        items.append(item)
        return item,201
    def delete(self,name):
        global items
        items = list(filter(lambda x: x['name'] != name,items))
        return {'message':"item deleted"}
    
class ItemList(Resource):
    
    def get(self):
        return {"item":items}
    def post(self):
        data = request.get_json(force = True)
        item = {"name":data["name"],"price":data["price"]}
        items.append(item)
        return item
    
api.add_resource(Item,"/item/<string:name>")
api.add_resource(ItemList,"/items")

app.run(port=1100)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:1100/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Jun/2020 09:11:16] "GET /items HTTP/1.1" 200 -
